In [287]:
#Imports

import re
import collections
import bisect
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import RegexpTokenizer
nltk.download('stopwords')
dados = pd.read_csv('https://raw.githubusercontent.com/Leonardomotta/lab06-ri/master/results.csv')
dados_json = pd.read_json('https://raw.githubusercontent.com/Leonardomotta/lab06-ri/master/results_final.json')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Leonardo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [288]:
resposta = {dados_json['query'][j]:dados_json['docs'][j] for j in range(10)}

tknz = RegexpTokenizer(r'([A-Za-zÁáÉéÍíÓóÚúÃãÕõÇçÂâÊê]{3,27})')
i = {}
n = 0
stopwords = nltk.corpus.stopwords.words('portuguese')



for texto in dados.text:
  words = [word for word in tknz.tokenize(texto.lower())
           if not bool(re.search(r'\d', word))
           and word not in stopwords and len(word) >= 3]  
  n += 1
  for j in words:
    if j not in i.keys():
      i[j] = []
    i[j].append(n)
    
for elementos in i.items():
  d = dict(collections.Counter(elementos[1]))
  i[elementos[0]] = list(d.items())
    
M = dados.text.count()

    
    
for w in i:
  k = len(i[w])
  IDF = round(np.log((M+1)/k),2)
  i[w].append(IDF)

In [298]:
#1 
#Escolha um documento dentre aqueles da base do aluno Bernardi e 
#crie uma consulta que você acha que tem boas chances de recuperar este documento. 
#Em seguida, avalie os resultados de tal consulta usando a métrica de avaliação Reciprocal Rank
numero_doc = 246
document = dados.loc[numero_doc]
query = 'aviões russos Venezuela militares     '




In [290]:


def tfidf_vsm(query, document):
  s = 0
  doc_tokens = document.split()
  query_tokens = query.split()
  
  for word in query_tokens:
    cwd = doc_tokens.count(word)
    if word in i:
      s += cwd * i[word][-1]
  
  return round(s,2)

def binary_vsm(query, document):
  s = 0
  query_tokens = query.split()
  doc_tokens = document.split()
  
  for token in query_tokens:
    s += (token in doc_tokens)
    
  return s

def tf_vsm(query, document):
  s = 0
  doc_tokens = document.split()
  query_tokens = query.split()
  
  for word in query_tokens:
    s += doc_tokens.count(word)
  
  return s


def bm25_vsm(query, document, k):
  s = 0
  doc_tokens = document.split()
  query_tokens = query.split()
  
  words = [word for word in query_tokens if word in doc_tokens]
    
  for word in words:
    cwd = doc_tokens.count(word)
    dfw = 0
    if word in i:
      dfw = len(i[word][:-1])
    s += (((k+1) * cwd) / (cwd + k)) * np.log10(((M+1) / dfw)) if dfw != 0 else 0
  
  return round(s,2)

def topN_models(q,n):
  db = []
  dtf = []
  dtfidf = []
  dbm25 = []
    
  for i in range(len(dados)):
   
    bisect.insort(dtfidf, (tfidf_vsm(query,dados.text[i].lower()), i))
    bisect.insort(db, (binary_vsm(query, dados.text[i].lower()), i))
    bisect.insort(dtf, (tf_vsm(query,dados.text[i].lower()), i))
    bisect.insort(dbm25, (bm25_vsm(query,dados.text[i].lower(),20), i))
  
  db.reverse()
  dtf.reverse()
  dtfidf.reverse()
  dbm25.reverse()
  
  return db[:n], dtf[:n], dtfidf[:n], dbm25[:n]

binary, tf, tfidf, bm25 = topN_models(query,10)


idc= [d for score,d in top_binary]



dataFrame = pd.DataFrame()

dataFrame['Binary'] = binary
dataFrame['BM25'] = bm25
dataFrame['TF'] = tf
dataFrame['TF-IDF'] =tfidf
dataFrame.index+=1
dataFrame




,Binary,BM25,TF,TF-IDF
1,"(3, 246)","(10.28, 246)","(10, 207)","(28.98, 142)"
2,"(2, 125)","(9.78, 142)","(9, 246)","(26.24, 246)"
3,"(1, 245)","(9.1, 125)","(8, 64)","(24.46, 125)"
4,"(1, 237)","(5.73, 207)","(7, 223)","(18.8, 207)"
5,"(1, 232)","(4.91, 64)","(7, 142)","(15.04, 64)"
6,"(1, 229)","(4.45, 223)","(7, 125)","(13.16, 223)"
7,"(1, 228)","(3.44, 103)","(5, 103)","(9.4, 103)"
8,"(1, 223)","(2.86, 168)","(4, 168)","(7.52, 168)"
9,"(1, 222)","(2.86, 165)","(4, 165)","(7.52, 165)"
10,"(1, 215)","(2.86, 114)","(4, 114)","(7.52, 114)"


In [291]:
def rankR(t, id_doc):
  x = 1.0
  for r,doc in t:
    if doc == id_doc:
      return  [round(1 / x, 2)]
    else:
        x += 1

rdf = pd.DataFrame()
rdf['Binary'] = rankR(dataFrame['Binary'], numero_doc)

rdf['BM25'] = rankR(dataFrame['BM25'], numero_doc)

rdf['TF'] = rankR(dataFrame['TF'], numero_doc)

rdf['TF-IDF'] = rankR(dataFrame['TF-IDF'], numero_doc)

rdf.index+=1
rdf

,Binary,BM25,TF,TF-IDF
1,1.0,1.0,0.5,0.5


In [292]:
#2
#A partir do gabarito fornecido em OBS1, calcule 
#o MAP para cada algoritmo abaixo e aponte qual obteve o melhor resultado. 
#Para os cálculos do MAP, considere que um documento é relevante 
#para uma dada consulta se este documento estiver entre os documentos
#do gabarito para essa consulta, senão ele deve ser considerado irrelevante. (10 pts)



In [293]:

def get_indexes(docs):
  return [d for score,d in docs]

def getIntersection(a,b):
    return [elem for elem in a if elem in b]
def cap(qr):
  doc_rel = []

  for doc_info in resposta[qr]:
    row = dados.loc[dados.url == doc_info['URL']]
    doc_rel.append(row.index[0])

  binary, tf, tfidf, bm25 = topN_models(qr, 10)
  binary = get_indexes(binary)
  tf = get_indexes(tf)
  tfidf = get_indexes(tfidf)
  bm25 = get_indexes(bm25)

  
  ap_binary = len(getIntersection(binary, doc_rel)) / len(binary)
  
  ap_tf = len(getIntersection(tf, doc_rel)) / len(tf)

  ap_tfidf = len(getIntersection(tfidf, doc_rel)) / len(tfidf)

  ap_bm25 = len(getIntersection(bm25, doc_rel)) / len(bm25)
  
  return ap_binary, ap_tf, ap_tfidf, ap_bm25
  
def cmap(qr):
  x = 0
  z = 0
  w = 0
  y = 0
  
  for q in qr:
    ap_binary, ap_tf, ap_tfidf, ap_bm25 = cap(q)
    
    x +=ap_binary
    z +=ap_tf
    w +=ap_tfidf
    y +=ap_bm25
  mb = round(x /   len(qr),2)
  mtfi = round(w / len(qr),2)
  mbm = round(y / len(qr),2)
  mtf = round(z / len(qr),2)
  
  return mb, mtf, mtfi, mbm


mapResult_binary, mapResult_tf, mapResult_tfidf, mapResult_bm25 = cmap(resposta.keys())




In [294]:
mapResult_binary

0.02

In [295]:
mapResult_tf

0.03

In [296]:
mapResult_tfidf

0.03

In [297]:
mapResult_bm25

0.03